In [5]:
import sys, os, time, shutil, random
from pathlib import Path
# if '_cwd' not in locals():
#     _cwd = os.getcwd()
#     os.chdir(Path(_cwd)/'..')

import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from tabulate import tabulate
import matplotlib.patches as mpatches
from tqdm.notebook import tqdm
from copy import deepcopy

from omnibelt import load_yaml, save_yaml


In [2]:
root = Path('data/classic/')


In [15]:
nodes = load_yaml(root / 'nodes.yaml')
edges = load_yaml(root / 'edges.yaml')
pos = load_yaml(root / 'pos.yaml')
len(nodes), len(edges), len(pos)

(75, 2, 122)

In [6]:
graph = deepcopy(nodes)
bgs = {}
len(graph)

75

In [11]:
for ID, es in edges['army'].items():
    node = graph[ID]
    if 'edges' not in node:
        node['edges'] = {'army': es.copy()}


In [12]:
for ID, es in edges['fleet'].items():
    node = graph.get(ID)
    coast = None
    if node is None:
        base, coast = ID.split('-')
        node = graph[base]
    if 'edges' not in node:
        node['edges'] = {}
    if coast is None:
        node['edges']['fleet'] = []
        ne = node['edges']['fleet']
    elif 'fleet' in node['edges']:
        node['edges']['fleet'][coast] = []
        ne = node['edges']['fleet'][coast]
    else:
        node['edges']['fleet'] = {coast:[]}
        ne = node['edges']['fleet'][coast]
    ne.extend(es)

In [19]:
for ID, pois in pos.items():
    node = graph.get(ID)
    coast = None
    if node is None:
        base, coast = ID.split('-')
        if base == 'neutral':
            bgs[coast] = deepcopy(pois['fill'])
            continue
        node = graph[base]
    
    if 'locs' not in node:
        node['locs'] = {}
    locs = node['locs']
    if coast is None:
        locs['unit'] = [pois['base'].copy()]
        if 'center' in pois:
            locs['sc'] = [pois['center'].copy()]
        if 'retreat' in pois:
            locs['retreat'] = [pois['retreat'].copy()]
        if 'text' in pois:
            locs['label'] = [pois['text'].copy()]
    elif coast == 'c':
        locs['coast-unit'] = [pois['base'].copy()]
        if 'retreat' in pois:
            locs['coast-retreat'] = [pois['retreat'].copy()]
    else:
        if 'coast-unit' not in locs:
            locs['coast-unit'] = {}
        locs['coast-unit'][coast] = [pois['base'].copy()]
        if 'retreat' in pois:
            if 'coast-retreat' not in locs:
                locs['coast-retreat'] = {}
            locs['coast-retreat'][coast] = [pois['retreat'].copy()]
        if 'text' in pois:
            if 'coast-label' not in locs:
                locs['coast-label'] = {}
            locs['coast-label'][coast] = [pois['text'].copy()]
            
    if 'fill' in pois:
        node['region-coords'] = deepcopy(pois['fill'])

In [23]:
graph['rom']['locs']

{'unit': [[957.1212823716937, 1627.102511094065]],
 'sc': [[921.7425647433879, 1567.400925096299]],
 'retreat': [[904.053205929235, 1580.6679442069135]],
 'label': [[937.2207537057716, 1593.9349633175282]],
 'coast-unit': [[946.065433112848, 1627.102511094065]],
 'coast-retreat': [[928.3760742986951, 1638.1583603529104]]}

In [24]:
# save_yaml(graph, root / 'graph.yaml')

In [25]:
# save_yaml(bgs, root / 'bgs.yaml')